In [ ]:
def tokenize(df_col):
    all_tokenized = []
    for i in range(len(df_col)):
        word_punct_token = WordPunctTokenizer().tokenize(data_preprocessed.iloc[i])
        clean_token=[]
        for token in word_punct_token:
            token = token.lower()
            # remove any value that are not alphabetical
            new_token = re.sub(r'[^a-zA-Z]+', '', token) 
            # remove empty value and single character value
            if new_token != "" and len(new_token) >= 2: 
                vowels=len([v for v in new_token if v in "aeiou"])
                if vowels != 0: # remove line that only contains consonants
                    clean_token.append(new_token)
        all_tokenized.append(clean_token)
    
    return all_tokenized

def remove_stop_words(arr):
    # Get the list of stop words
    stop_words = stopwords.words('english')
    # add new stopwords to the list
    stop_words.extend(["could","though","would","also","many",'much'])
    # print(stop_words)
    # Remove the stopwords from the list of tokens
    tokens = [x for x in arr if x not in stop_words]
    return tokens

def token_to_text(tokens):
    text = ""
    for word in tokens:
        text = text + " " + word
    
    return text

def count_probabilities(text, length):
    vectorizer = CountVectorizer()
    vector = vectorizer.fit_transform([text])
    vector = vector.toarray()
    vector = vector[0]/length
    vocab = vectorizer.get_feature_names()
    result_dictionary = dict(zip(vocab, vector))
    return result_dictionary

def get_global_vocab(all_tokens):
    global_vocab = []
    for row in all_tokens:
        for word in row:
            global_vocab.append(word)
    return global_vocab

def get_unique_global_vocab(all_tokens):
    global_vocab = []
    for row in all_tokens:
        for word in row:
            global_vocab.append(word)
    unique_vocab = list(set(global_vocab))
    
    return unique_vocab

def get_global_dictionary(global_vocab):
    dic = {}
    for word in global_vocab:
        if word in dic:
            dic[word] += 1
        else:
            dic[word] = 0
    return dic

    

In [ ]:
import re
from nltk.tokenize import WordPunctTokenizer
import html2text
import pandas as pd
from nltk import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
import textcleaner as tc
import numpy as np
import matplotlib.pyplot as plt
import nltk
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score
from sklearn.metrics import multilabel_confusion_matrix

In [ ]:
train_df = pd.read_csv("untag.csv")
test_df = pd.read_csv("untag_test.csv")
train_df = train_df.sample(frac=1, axis=1).sample(frac=1).reset_index(drop=True)

In [ ]:
df = pd.concat([train_df,test_df],ignore_index=True)

In [ ]:
train_df.Source.value_counts();

###  Balancing Data

In [ ]:
df[df.Source == "Businessweek"]
df = df.drop(1115)
df = df.drop(804)
df = df.drop(1433)
df = df.drop(1949)
df = df.drop(2445)
df = df.drop(1925)

In [ ]:
df[df.Source == "Contactmusic.com"]
df = df.drop(99)
df = df.drop(400)
df = df.drop(1192)
df = df.drop(1774)
df = df.drop(2079)
df = df.drop(994)

In [ ]:
df[df.Source == "Forbes"]
df = df.drop(949)
df = df.drop(963)
df = df.drop(1016)
df = df.drop(1093)
df = df.drop(2680)
df = df.drop(1466)

In [ ]:
df = df.reset_index(drop=True)

In [ ]:
data_preprocessed = df["Tag Removed"]

In [ ]:
tokenized = tokenize(data_preprocessed)

In [ ]:
# df = df.reset_index(drop=True)

In [ ]:
stop_words_removed = []
for arr in tokenized:
    stop_words_removed.append(remove_stop_words(arr))    

In [ ]:
unique_global_vocab = get_unique_global_vocab(stop_words_removed)

In [ ]:
global_dict = {}
temp = get_global_vocab(stop_words_removed)

x = get_global_dictionary(temp)

all_words = dict(sorted(x.items(), key=lambda item: item[1],reverse=True))


In [ ]:
# for key,value in all_words.items():
#     if value > 0 and value < 2:
#         print(key,end="|")

In [ ]:
final_dict = {}
final_vocab = []

for key,value in all_words.items():
    if value < 3000 and value > 5:
        if key not in final_dict:
            final_dict[key] = value
            final_vocab.append(key)
    

In [ ]:
final_stop_words_removed = []

for row in stop_words_removed:
    temp = []
    for word in row:
        if word in final_dict:
            temp.append(word)
    final_stop_words_removed.append(temp)
    

In [ ]:
zero_arr = np.zeros((len(df),len(final_dict)))

In [ ]:
word_df = pd.DataFrame(data=zero_arr, columns=final_vocab)

In [ ]:
text_array = []

for row in final_stop_words_removed:
    text_array.append(token_to_text(row))


In [ ]:
count = 0
for index in range(len(df)):
    if(len(final_stop_words_removed[index])>0):
        word_probabilities = count_probabilities(text_array[index],len(final_stop_words_removed[index]))
        for key,value in word_probabilities.items():
            word_df.loc[index,key] = word_probabilities[key]
    count += 1
    if count > 200:
        print(index,end="->")
        count = 0

200->401->602->803->1004->1205->1406->1607->1808->2009->2210->2411->2612->2813->3014->3215->3416->

In [ ]:
df.head(2732)
num = 2705

## Model

In [ ]:
X = word_df.head(num)
y = df["Source"].head(num)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
model = RandomForestClassifier(n_estimators=200,oob_score=True, n_jobs = 3,
                               bootstrap=True, random_state=1)

In [ ]:
model.fit(X,y)

RandomForestClassifier(n_estimators=200, n_jobs=3, oob_score=True,
                       random_state=1)

In [ ]:
model.score(X_test,y_test)

## Save Results

In [ ]:
predictions = model.predict(word_df.tail(688))

In [ ]:
ids = list(range(1,689))

In [ ]:
new_df = pd.DataFrame()

In [ ]:
new_df["Id"] = ids

In [ ]:
new_df["Publisher"] = predictions

In [ ]:
new_df.to_csv("results_3.csv",index=False)